In [1]:
from datasets import load_dataset

In [3]:
import pandas as pd

df_train = pd.DataFrame(dataset["train"])
df_val = pd.DataFrame(dataset["validation"])
df_test = pd.DataFrame(dataset["test"])

In [4]:
# Save to CSV files
df_train.to_csv("pubmed_train.csv", index=False)
df_val.to_csv("pubmed_validation.csv", index=False)
df_test.to_csv("pubmed_test.csv", index=False)

print("CSV files saved successfully!")

CSV files saved successfully!


In [5]:
df = pd.read_csv("pubmed_train.csv")
df.head()  # Show first few rows

,article,abstract
0,a recent systematic analysis showed that in 20...,background : the present study was carried out...
1,it occurs in more than 50% of patients and may...,backgroundanemia in patients with cancer who a...
2,"tardive dystonia ( td ) , a rarer side effect ...",tardive dystonia ( td ) is a serious side effe...
3,"lepidoptera include agricultural pests that , ...",many lepidopteran insects are agricultural pes...
4,syncope is caused by transient diffuse cerebra...,we present an unusual case of recurrent cough ...


In [6]:
import pandas as pd

# Load CSV files (Assuming you have saved them as pubmed_train.csv, pubmed_validation.csv, pubmed_test.csv)
df_train = pd.read_csv("pubmed_train.csv")
df_val = pd.read_csv("pubmed_validation.csv")
df_test = pd.read_csv("pubmed_test.csv")

# Display first few rows
df_train.head()

,article,abstract
0,a recent systematic analysis showed that in 20...,background : the present study was carried out...
1,it occurs in more than 50% of patients and may...,backgroundanemia in patients with cancer who a...
2,"tardive dystonia ( td ) , a rarer side effect ...",tardive dystonia ( td ) is a serious side effe...
3,"lepidoptera include agricultural pests that , ...",many lepidopteran insects are agricultural pes...
4,syncope is caused by transient diffuse cerebra...,we present an unusual case of recurrent cough ...


In [7]:
print(df_train.columns)

Index(['article', 'abstract'], dtype='object')


In [8]:
import pandas as pd
import json

# Step 1: Load the arXiv dataset (JSON format)
arxiv_file = "arxiv-metadata-oai-snapshot.json"  # Update with your actual file path

# Read the JSON file line by line (since it's in JSONL format)
data = []
with open(arxiv_file, "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))

# Convert to DataFrame, ensuring columns exist
arxiv_df = pd.DataFrame(data, columns=["id", "title", "abstract", "categories"])

# Save as CSV
arxiv_df.to_csv("arxiv-dataset.csv", index=False, encoding="utf-8")

print("ArXiv Dataset Successfully Converted to CSV!")
print(arxiv_df.head())  # Display first few rows

ArXiv Dataset Successfully Converted to CSV!
          id                                              title  \
0  0704.0001  Calculation of prompt diphoton production cros...   
1  0704.0002           Sparsity-certifying Graph Decompositions   
2  0704.0003  The evolution of the Earth-Moon system based o...   
3  0704.0004  A determinant of Stirling cycle numbers counts...   
4  0704.0005  From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...   

                                            abstract       categories  
0    A fully differential calculation in perturba...           hep-ph  
1    We describe a new algorithm, the $(k,\ell)$-...    math.CO cs.CG  
2    The evolution of Earth-Moon system is descri...   physics.gen-ph  
3    We show that a determinant of Stirling cycle...          math.CO  
4    In this paper we show how to compute the $\L...  math.CA math.FA  


In [ ]:
import pandas as pd
import json
from transformers import pipeline

# Load arXiv dataset
arxiv_file = "arxiv-metadata-oai-snapshot.json"

with open(arxiv_file, "r", encoding="utf-8") as f:
    arxiv_data = [json.loads(line) for line in f]
    arxiv_df = pd.DataFrame(arxiv_data)

arxiv_df = arxiv_df[["id", "title", "abstract", "categories"]]
print("Loaded arXiv dataset!", arxiv_df.shape)

# Load CompScholar dataset
comp_scholar_file = "/mnt/data/Brain Dead CompScholar Dataset.csv"
comp_scholar_df = pd.read_csv(comp_scholar_file)
print("Loaded CompScholar dataset!", comp_scholar_df.shape)

# Load PubMed dataset
pubmed_file = "/mnt/data/pubmed_test.xls"
pubmed_df = pd.read_excel(pubmed_file)
print("Loaded PubMed dataset!", pubmed_df.shape)

# Combine datasets for summarization
combined_df = pd.concat([arxiv_df, comp_scholar_df, pubmed_df], ignore_index=True)
print("Combined dataset shape:", combined_df.shape)

# Summarization using a pretrained model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def generate_summary(text):
    try:
        summary = summarizer(text[:1024], max_length=200, min_length=50, do_sample=False)
        return summary[0]['summary_text']
    except Exception as e:
        return "Error in summarization"

combined_df["summary"] = combined_df["abstract"].apply(lambda x: generate_summary(str(x)))

# Save summarized dataset
combined_df.to_csv("summarized_research_articles.csv", index=False)
print("Summarization complete! File saved as summarized_research_articles.csv")
